GWR (Geographically Weighted Regression) is a spatially varying coefficients model that explores how relationships between predictors (independent variables) and an outcome (dependent variable) change across space.

In this project we want to investigate how mortality counts from pulmonary embolism changes over space (counties)

In [24]:
import os
import configparser
import time
import warnings
import numpy as np
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
import statsmodels.api as sm
from mgwr.gwr import GWR, MGWR
from mgwr.sel_bw import Sel_BW
from mgwr.utils import shift_colormap
from shapely import wkt
from shapely.geometry import Point
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
warnings.filterwarnings('ignore')

In [115]:
BASE_DIR = os.getcwd()
CONFIG = configparser.ConfigParser()
CONFIG.read(os.path.join(BASE_DIR, 'script_config.ini'))

BASE_PATH = os.path.abspath(os.path.join(os.getcwd(), '..', 'data'))

DATA_RAW = os.path.join(BASE_PATH, 'raw')
DATA_RESULTS = os.path.join(BASE_PATH, '..', 'results')

In [140]:
data_path = os.path.join(DATA_RESULTS, 'final', 
                   'pulmonary_air_quality_data.csv')

df = pd.read_csv(data_path)
df = df[df['geometry'].notna()]
df = gpd.GeoDataFrame(df, geometry = 
     gpd.GeoSeries.from_wkt(df['geometry']), crs = 'EPSG:4326')

df = df[['geometry', 'sex', 'race_recode3', 'age_cat', 
         'Daily Mean PM2.5 Concentration', 'Daily AQI Value', 
         'mort_per_100k']]

In [141]:
df = (df.groupby(['geometry', 'Daily Mean PM2.5 Concentration', 
      'Daily AQI Value'], as_index = False)['mort_per_100k'].sum())
df = df.drop('Daily Mean PM2.5 Concentration', axis=1)

However, GWR requires numeric predictors.
We have to converg the categorical variables in our dataset into numeric dummy variables (one-hot encoding). But before we do that, we first have to handle the missing values.

We fill the missing values with the most frequent.

In [75]:
cat_cols = ['age_cat']
for col in cat_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

Next we encode the categorical columns with dummy variables

In [76]:
df_encoded = pd.get_dummies(df, columns = 
            ["age_cat"], 
             dtype = int, drop_first = True)

Now, we define our dependent (y) variable i.e mort_per_100k and independent (x) variables ('Daily Mean PM2.5 Concentration', 'Daily AQI Value').

In [119]:
y = df["mort_per_100k"].values.reshape((-1, 1))

In [120]:
X = df[["Daily AQI Value"]] 

In [121]:
# Only convert to shapely geometry if it's not already one
def ensure_geometry(val):
    if isinstance(val, str):
        return wkt.loads(val)
    elif isinstance(val, Point):
        return val
    else:
        return None 

In [122]:
df['geometry'] = df['geometry'].apply(ensure_geometry)
df = gpd.GeoDataFrame(df, geometry='geometry')

Now we extract individual (x, y) coordinates from the geometry column.

In [123]:
u = df.geometry.x.values
v = df.geometry.y.values
coords = np.column_stack((u, v))

In [124]:
df

,geometry,Daily AQI Value,mort_per_100k
0,POINT (-274851.958 393465.259),25.534483,45.703839
1,POINT (-274851.958 393465.259),28.095101,28.650620
2,POINT (-274851.958 393465.259),30.563218,31.426776
3,POINT (-274851.958 393465.259),32.270195,30.373339
4,POINT (-274851.958 393465.259),37.015748,25.460497
...,...,...,...
84503,POINT (1494890.563 697303.595),46.159884,15.207959
84504,POINT (1494890.563 697303.595),45.839833,12.034716
84505,POINT (1494890.563 697303.595),45.157576,38.738558
84506,POINT (1494890.563 697303.595),46.249315,81.412507


We now fit the GWR model.

In [125]:
sample_size = min(5000, len(df))
sample_idx = np.random.choice(len(df), sample_size, replace=False)
selector = Sel_BW(coords[sample_idx], y[sample_idx], X_scaled[sample_idx])
bw = selector.search(bw_min=2)

In [126]:
start_time = time.perf_counter()

gwr_model = GWR(coords, y, X_scaled, bw=bw, fixed=False, kernel='bisquare')
gwr_results = gwr_model.fit()
print(gwr_results.summary())

end_time = time.perf_counter()
fit_duration = end_time - start_time
print(f"GWR model fitted in: {fit_duration:.4f} seconds.")

/Users/bosoro/anaconda3/envs/geosp/lib/python3.12/site-packages/mgwr/kernels.py:60: RuntimeWarning: divide by zero encountered in divide
  self.kernel = self._kernel_funcs(self.dvec / self.bandwidth)
/Users/bosoro/anaconda3/envs/geosp/lib/python3.12/site-packages/mgwr/kernels.py:60: RuntimeWarning: divide by zero encountered in divide
  self.kernel = self._kernel_funcs(self.dvec / self.bandwidth)
/Users/bosoro/anaconda3/envs/geosp/lib/python3.12/site-packages/mgwr/kernels.py:60: RuntimeWarning: divide by zero encountered in divide
  self.kernel = self._kernel_funcs(self.dvec / self.bandwidth)
/Users/bosoro/anaconda3/envs/geosp/lib/python3.12/site-packages/mgwr/kernels.py:60: RuntimeWarning: invalid value encountered in divide
  self.kernel = self._kernel_funcs(self.dvec / self.bandwidth)
/Users/bosoro/anaconda3/envs/geosp/lib/python3.12/site-packages/mgwr/kernels.py:60: RuntimeWarning: invalid value encountered in divide
  self.kernel = self._kernel_funcs(self.dvec / self.bandwidth)
/U

LinAlgError: Matrix is singular.

Any constant columns? False
Optimal bandwidth: 652.0
Code block executed in: 59.4893 seconds.


Now we extract the local parameter estimates (coefficients), R² and residuals.

In [34]:
local_coefs = results.params 
local_R2 = results.localR2  
residuals = results.resid_response

We then combine these arrays with our  original data and geometry:

In [36]:
predictor_names = ["Intercept"] + list(
    ["Daily Mean PM2.5 Concentration", "Daily AQI Value"]
    + [col for col in df_encoded.columns if col.startswith((
        "sex_", "race_recode3_", "age_cat_"))])

coef_df = pd.DataFrame(local_coefs, columns = predictor_names)

coef_df["local_R2"] = local_R2
coef_df["residuals"] = residuals

gwr_df = gpd.GeoDataFrame(pd.concat([df_encoded.reset_index(
    drop = True), coef_df], axis = 1), geometry = df_encoded.geometry,
                           crs = df_encoded.crs)

In [38]:
folder_out = os.path.join(DATA_RESULTS, 'final')

filename = 'GWR_results.csv'
path_out = os.path.join(folder_out, filename)
gwr_df.to_csv(path_out, index = False)